In [ ]:
import shutil
import matplotlib.pyplot as plt
from sklearn.utils import resample
import numpy as np 
import pandas as pd 
import os 
from os import listdir
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.utils import np_utils
from sklearn.metrics import accuracy_score
%matplotlib inline
#************************************************

In [ ]:
os.mkdir('augmented')
os.mkdir('/kaggle/working/augmented/benign')
os.mkdir('/kaggle/working/augmented/malignant')


In [ ]:
def getListOfFiles(dirName):
    listOfFile = os.listdir(dirName)
    allFiles = list()
    for entry in listOfFile:
        fullPath = os.path.join(dirName, entry)
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles



In [ ]:
files_benign=getListOfFiles('../input/breakhis/BreaKHis_v1/BreaKHis_v1/histology_slides/breast/benign')
for f in files_benign:
    if f.endswith('.png'):
        
        shutil.copy(f,'augmented/benign')
files_malignant=getListOfFiles('../input/breakhis/BreaKHis_v1/BreaKHis_v1/histology_slides/breast/malignant')
for f in files_malignant:
    if f.endswith('.png'):
        
        shutil.copy(f,'augmented/malignant')

In [ ]:
benign=getListOfFiles('/kaggle/working/augmented/benign')
malignent=getListOfFiles('/kaggle/working/augmented/malignant')


In [ ]:
image.load_img(benign[0], target_size=(120,120,1), grayscale=False)
    

In [ ]:

image.load_img(malignent[0], target_size=(120,120,1), grayscale=False)
    

In [ ]:
len(benign)+len(malignent)

In [ ]:
data = pd.DataFrame(index=np.arange(0, len(benign)+len(malignent)), columns=["image", "target"])
k=0
for c in [0,1]:
        if c==1:
            for m in range(len(benign)):
                data.iloc[k]["image"] = benign[m]
                data.iloc[k]["target"] = 0
                k += 1
        else:
            for m in range(len(malignent)):
                data.iloc[k]["image"] = malignent[m]
                data.iloc[k]["target"] = 1
                k += 1
                


In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
count=data["target"].value_counts() 
count


In [ ]:
import seaborn as sns
'''
Visualize the target variable
'''

target=sns.countplot(data['target'])
target.set_xticklabels(['0',' 1' ])
plt.show()

In [ ]:
mal_downsampled = resample(data[data['target']==1],n_samples=data[data['target']==0].shape[0], random_state=42)

downsampled = pd.concat([data[data['target']==0], mal_downsampled])

downsampled['target'].value_counts()

In [ ]:
downsampled.head()

In [ ]:
downsampled.shape

In [ ]:
train_image = []
y = []

for i in tqdm(range(downsampled.shape[0])):
    img = image.load_img(downsampled['image'].iloc[i], target_size=(28,28,1), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)

        
X = np.array(train_image)
y = downsampled.iloc[:,-1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, random_state=42, test_size=0.2 , shuffle=True)

Y_train = np_utils.to_categorical(y_train, 2)
Y_test = np_utils.to_categorical(y_test, 2)
Y_val = np_utils.to_categorical(y_val, 2)

print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

In [ ]:
model = Sequential()

model.add(Conv2D(30, kernel_size = (5,5), strides=(1,1), padding='valid', activation='relu', input_shape=(28,28,3)))

model.add(MaxPool2D(pool_size=(3,3)))

model.add(Flatten())   # flatten output of conv
model.add(Dense(150, activation='relu'))  # hidden layer of 150 neuron
model.add(Dense(2, activation='softmax'))  # output layer
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

history = model.fit(X_train, Y_train, batch_size=10, epochs = 10, validation_data=(X_test, Y_test))

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.plot()

In [ ]:
y_pred = model.predict_classes(X_val)
acc_test = 0

for i in range(X_val.shape[0]):
    if(y_pred[i] == y_val[i]):
        acc_test= acc_test+1
print("Accuracy test : "  , acc_test/X_val.shape[0]*100)